In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install modelscope==1.4.2
!pip install open_clip_torch
!pip install pytorch-lightning

In [ ]:
cd modelscope

/content/modelscope


In [ ]:
from huggingface_hub import snapshot_download
from modelscope.pipelines import pipeline
from modelscope.outputs import OutputKeys
import pathlib

model_dir = pathlib.Path('weights')
snapshot_download('damo-vilab/modelscope-damo-text-to-video-synthesis',
                   repo_type='model', local_dir=model_dir)

In [ ]:
pipe = pipeline('text-to-video-synthesis', model_dir.as_posix())

In [ ]:
prompt_dict = {'text' : 'a panda eating bamboo on a rock.'}
output_video_path = pipe(prompt_dict,)[OutputKeys.OUTPUT_VIDEO]

In [ ]:
!mv ... /modelscope/output/

In [ ]:
from shutil import move

def render_prompt_n_times(prompt, pipe, outdir, n=1):
    for i in range(n):
        print('Rendering video number '+str(i)+' ...')
        output_video_path = pipe({'text':prompt},)[OutputKeys.OUTPUT_VIDEO]
        move(output_video_path, outdir+str(i)+'.mp4')

In [1]:
# Preprocessing textfiles

def get_sentences_from_file(filepath, repeats_per_prompt=1):
    with open(filepath, 'r') as f:
        txt = f.read()
        # Clean Zeilenumbrüche
        txt_ = ""
        i = 0
        while i < len(txt):
            if txt[i] == '\n' or txt[i] == "\'":
                i += 1
            else:
                txt_ += txt[i]
                i += 1

        # To prompts
        prompts = []; prompt = ""; i=0
        while i < len(txt_):
            if txt_[i] == " " and len(prompt) == 0:
                i += 1
                continue
            if txt_[i] == '.':
                prompt += '.'
                if i < len(txt_) - 1:
                  if txt_[i + 1] in ("'", '"'):
                      i += 1
                      prompt += txt_[i]
                for j in range(repeats_per_prompt):
                    prompts.append(prompt)
                prompt = ""
            else:
                prompt += txt_[i]
            i += 1
        return prompts

In [3]:
prompts = get_sentences_from_file("txt.txt")

In [4]:
prompts

['Once upon a time there was a sweet little girl.',
 'Everyone who saw her liked her, but most of all her grandmother, who did not know what to give the child next.',
 'Once she gave her a little cap made of red velvet.',
 'Because it suited her so well, and she wanted to wear it all the time, she came to be known as Little Red Riding Hood.',
 'One day her mother said to her: "Come Little Red Riding Hood.',
 'Here is a piece of cake and a bottle of wine.',
 'Take them to your grandmother.',
 'She is sick and weak, and they will do her well.',
 'Mind your manners and give her my greetings.',
 'Behave yourself on the way, and do not leave the path, or you might fall down and break the glass, and then there will be nothing for your sick grandmother."',
 'Little Red Riding Hood promised to obey her mother.',
 'The grandmother lived out in the woods, a half hour from the village.',
 'When Little Red Riding Hood entered the woods a wolf came up to her.',
 'She did not know what a wicked anim

In [5]:
def render_each_prompt(list_of_prompts, pipe, outdir):
    for i, prompt in enumerate(list_of_prompts):
        print('Rendering video number '+str(i)+' ...')
        output_video_path = pipe({'text':prompt},)[OutputKeys.OUTPUT_VIDEO]
        move(output_video_path, outdir+str(i)+'.mp4')

In [ ]:
render_each_prompt(
    prompts,
    pipe,
    '/content/modelscope/output/')

2023-04-17 12:51:13,372 - modelscope - WARNING - task text-to-video-synthesis input definition is missing


Rendering video number 0 ...


2023-04-17 12:52:15,868 - modelscope - WARNING - task text-to-video-synthesis output keys are missing
2023-04-17 12:52:15,871 - modelscope - WARNING - task text-to-video-synthesis input definition is missing


Rendering video number 1 ...


2023-04-17 12:53:17,095 - modelscope - WARNING - task text-to-video-synthesis output keys are missing
2023-04-17 12:53:17,098 - modelscope - WARNING - task text-to-video-synthesis input definition is missing


Rendering video number 2 ...


2023-04-17 12:54:18,732 - modelscope - WARNING - task text-to-video-synthesis output keys are missing


In [ ]:
%%capture
!pip install moviepy

In [ ]:
from moviepy.editor import *

def concatenate(video_clip_paths, output_path, method="compose"):
    """Concatenates several video files into one video file
    and save it to `output_path`. Note that extension (mp4, etc.) must be added to `output_path`
    `method` can be either 'compose' or 'reduce':
        `reduce`: Reduce the quality of the video to the lowest quality on the list of `video_clip_paths`.
        `compose`: type help(concatenate_videoclips) for the info"""
    # create VideoFileClip object for each video file
    clips = [VideoFileClip(c) for c in video_clip_paths]
    if method == "reduce":
        # calculate minimum width & height across all clips
        min_height = min([c.h for c in clips])
        min_width = min([c.w for c in clips])
        # resize the videos to the minimum
        clips = [c.resize(newsize=(min_width, min_height)) for c in clips]
        # concatenate the final video
        final_clip = concatenate_videoclips(clips)
    elif method == "compose":
        # concatenate the final video with the compose method provided by moviepy
        final_clip = concatenate_videoclips(clips, method="compose")
    # write the output video file
    final_clip.write_videofile(output_path)

In [ ]:
clips_dir = "/content/modelscope/output/"
concatenate([clips_dir+i for i in os.listdir(clips_dir) if i.endswith('mp4')], 'concatenated.mp4')